In [ ]:
!chmod +x ./setup.sh
!./setup.sh

In [ ]:
import boto3, sagemaker,os

In [ ]:
MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
PUBLIC_ACCOUNT_ID = '763104351884'

REGION = boto3.session.Session().region_name

MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'
PUBLIC_ECR_ENDPOINT =  f'{PUBLIC_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'

IMAGE_NAME = 'lampps'
TAG=':latest'

MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

# Public ECR ログイン
!aws ecr get-login-password --region ap-northeast-1 | docker login --username AWS --password-stdin {PUBLIC_ECR_ENDPOINT}

In [ ]:
%cd ./container
!docker stop $(docker ps -q)
!docker rm $(docker ps -q -a)
!docker rmi -f $(docker images -a -q)
!docker build -t {IMAGE_NAME}{TAG} .
%cd ../

In [ ]:
!docker run --gpus all -v {os.getcwd()}/src/:/test -d {IMAGE_NAME}{TAG} sh -c 'tail -f /dev/null'